In [2]:
from bs4 import BeautifulSoup
import urllib.request
import xml.etree.ElementTree as ET
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
from datetime import datetime
import os
import schedule
from flask import Flask, request, jsonify

In [38]:
# Create Polygon from coordinates function 
def create_polygon(coords):
    points = []
    for coord in coords.split():
        lon, lat, _ = coord.split(',')
        points.append((float(lon), float(lat)))
    return Polygon(points)

In [57]:
def sentinel_scraper(url, kml_str):
    # Import Sentinel URL
    r = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(r, "html.parser")
    # Extract KML links from URL
    kml_links = soup.find_all('a', href=lambda href:  kml_str in href.lower())
    # Create a list to store the kml_content
    kml_contents = []
    # Iterate over the links
    for kml_link in kml_links:
        # Retrieve the URL of each KML file
        kml_url = 'https://sentinels.copernicus.eu' + kml_link['href']
        # Open the KML file
        kml_response = urllib.request.urlopen(kml_url)
        # Read the content of the KML file
        kml_content = kml_response.read()
        # Append the kml_content to the list
        kml_contents.append(kml_content)
    
    # Parse the KML content
    root = ET.fromstring(kml_content)

    # Define the XML namespace
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Extract latitude, longitude, time, mode and polarisation 
    data = [] # will create a list of dictionaries. 
    # Each dictionary in the list represents an observation, and the keys of the dictionary become columns in the DataFrame.
    
    for placemark in root.findall('.//kml:Placemark', namespace):
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip()
        time_start = placemark.find('.//kml:begin', namespace).text
        time_end = placemark.find('.//kml:end', namespace).text
        mode = placemark.find('.//kml:Data[@name="Mode"]/kml:value', namespace).text
        if 's1a' in kml_url:
            polarisation = placemark.find('.//kml:Data[@name="Polarisation"]/kml:value', namespace).text
            # Append the data to the list
            data.append({'coordinates': coordinates, 'time_start': time_start, 'time_end': time_end,
                 'mode': mode, 'polarisation': polarisation})
        else:
            # Append the data to the list
            data.append({'coordinates': coordinates, 'time_start': time_start, 'time_end': time_end,'mode': mode})

    # Convert the data to a DataFrame
    df = pd.DataFrame(data)

    # Create geometry column 
    df['geometry'] = df['coordinates'].apply(create_polygon)

    # Create a GeoDataFrame with Polygon geometry
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Apply the filter for mode and polarisation, depending on s1 or s
    if 's1a' in kml_url:
        gdf_filtered = gdf[(gdf['mode'] == 'IW') & (gdf['polarisation'] == 'DV')]
    else: 
        gdf_filtered = gdf[(gdf['mode'] == 'NOBS')]

    # Format GeoDataFrame
    gdf_clean = gdf_filtered.drop('coordinates', axis=1)
    gdf_final = gdf_clean.drop('mode', axis=1)
    if 's1a' in kml_url:
        gdf_final = gdf_final.drop('polarisation', axis=1)
        gdf_final['satellite'] = 'Sentinel 1'
    elif 's2a' in kml_url:
        gdf_final['satellite'] = 'Sentinel 2A'
    else:                    
        gdf_final['satellite'] = 'Sentinel 2B'
    
    return(gdf_final)

In [58]:
# Define URLs and KML identifier strings
s1_url = 'https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1/observation-scenario/acquisition-segments'
s1_kml_str = 's1a_mp_user'
s2_url = 'https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/acquisition-plans'
s2a_kml_str = 's2a_mp_acq__kml'
s2b_kml_str = 's2b_mp_acq__kml'

In [62]:
# Function to combine GeoDataFrames
def combine_gdfs(s1_url, s1_kml_str, s2_url, s2a_kml_str, s2b_kml_str):
    s1_gdf = sentinel_scraper(s1_url, s1_kml_str)
    s2a_gdf = sentinel_scraper(s2_url, s2a_kml_str)
    s2b_gdf = sentinel_scraper(s2_url, s2b_kml_str)
    combined_gdf = pd.concat([s1_gdf, s2a_gdf, s2b_gdf], ignore_index=True)
    return combined_gdf

,time_start,time_end,geometry,satellite
0,2023-07-03T18:41:51,2023-07-03T18:46:49,"POLYGON ((-6.78783 0.17358, -8.00093 5.97537, ...",Sentinel 1
1,2023-07-03T18:49:18,2023-07-03T18:53:31,"POLYGON ((-12.42288 27.18453, -15.94338 42.181...",Sentinel 1
2,2023-07-03T18:57:45,2023-07-03T19:00:25,"POLYGON ((-20.84242 57.63423, -21.89285 60.103...",Sentinel 1
3,2023-07-03T19:32:14,2023-07-03T19:34:48,"POLYGON ((153.56373 -2.74127, 152.60515 -7.143...",Sentinel 1
4,2023-07-03T19:36:36,2023-07-03T19:42:29,"POLYGON ((149.94393 -18.54735, 148.20944 -25.2...",Sentinel 1
...,...,...,...,...
2643,2023-07-24T12:16:33.362,2023-07-24T12:17:09.442,"POLYGON ((-37.97705 -52.71060, -40.05117 -52.3...",Sentinel 2B
2644,2023-07-24T13:17:07.430,2023-07-24T13:24:09.566,"POLYGON ((49.40945 81.47978, 49.62018 82.78700...",Sentinel 2B
2645,2023-07-24T13:38:28.435,2023-07-24T13:53:30.435,"POLYGON ((-45.02756 13.65628, -46.32848 13.937...",Sentinel 2B
2646,2023-07-24T13:56:25.314,2023-07-24T13:57:08.610,"POLYGON ((-61.81859 -49.82583, -63.77341 -49.4...",Sentinel 2B


In [64]:
# Filter for only future times
# Get the current date and time
current_time = datetime.now()

# Convert the 'time_start' column to datetime objects
combined_gdf['time_start'] = pd.to_datetime(combined_gdf['time_start'])

# Filter out rows where the 'time_end' is greater than or equal to the current time
time_filtered_gdf = combined_gdf[combined_gdf['time_start'] >= current_time]


In [65]:
# Check whether the point is contained within upcoming images
# Create a Point object for the longitude and latitude you want to check
lon = 24.05284
lat = 20.10057
user_coord = Point(lon, lat)

# Check if the point is contained within any of the polygons in the filtered GeoDataFrame
coord_mask = time_filtered_gdf.geometry.contains(user_coord)

# Get the rows where the point is contained within the polygon(s)
final_gdf = time_filtered_gdf[coord_mask]

final_gdf

,time_start,time_end,geometry,satellite
2590,2023-07-23 08:45:53.181,2023-07-23T09:18:10.677,"POLYGON ((101.87505 81.21880, 99.48271 82.4818...",Sentinel 2B
